In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_target = create_aml_compute(ws, "cluster1_project", "STANDARD_D2_V2",
  vm_priority = "dedicated", min_nodes = 0, max_nodes = 3,
  idle_seconds_before_scaledown = NULL, admin_username = NULL,
  admin_user_password = NULL, admin_user_ssh_key = NULL,
  vnet_resourcegroup_name = NULL, vnet_name = NULL,
  subnet_name = NULL, tags = NULL, description = NULL)


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({'max_iter':choice(50,75,100,150),'C':uniform(0.2,0.7)})

# Specify a Policy
policy = BanditPolicy()

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn.run_config()

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, primary_metric_name='accuracy', primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=7, max_concurrent_runs=7, max_duration_minutes=10080, policy=policy, run_config=est, resume_from=None, resume_child_runs=None, pipeline=run_config)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hd_run = experiment.submit(hyperdrive_configuration)
RunDetails(hd_run).show()


In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hd_run.get_best_run_by_primary_metric()
joblib.dump(best_run, 'hyperdrive_best_model.sav')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
rom azureml.core import Workspace, Dataset

subscription_id = '643cff33-8102-4a9d-98b5-7123b2e7a429'
resource_group = 'normal'
workspace_name = 'ml_workspace'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='bank-marketing-tabular')
ds = dataset.to_pandas_dataframe()

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train ,y_train, x_test, y_test = train_test_split( x, y, test_size=0.33, random_state=42)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=x_train,
    label_column_name=y_train,
    n_cross_validations=5)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = experiment.submit(automl_config, show_output=True)


In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run = automl_run.get_best_run_by_primary_metric()
joblib.dump(best_run, 'automl_best_model.sav')